<a href="https://colab.research.google.com/github/NITIN9694/TensorFlow-Learning/blob/main/0_8_Food_Vision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MileStone  Project 1 : Food Vision

##What we're going to cover


* Using TensorFlow Datasets to download and explore data
* Batching & preparing datasets for modelling (making our datasets run fast)
* Creating modelling callbacks
* Setting up mixed precision training
* Building a feature extraction model (see transfer learning part 1: feature   extraction)
*  Fine-tuning the feature extraction model (see transfer learning part 2: fine-tuning)
*  Viewing training results on TensorBoard









##Check GPU
Google collab offer free GPU

Google collab offers:
* K80 (not compatible)
* P100 (not compatible)
* Tesla T4 (not compatible)

Knowing this, in order to use mixed precision training we need access to Tesla T4 or if we're using own hardware , Our Gpu need a score of 7.0+

In [1]:
# Get GPU name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2859378a-30a7-40e6-a435-e6cfa3e90689)


In [2]:
#Get helper funcation
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-10-02 10:20:06--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-10-02 10:20:06 (21.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#Import series of helper funcation
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

#Use Tensorflow Dataset to Download Data
more info (https://www.tensorflow.org/datasets/dataset_collections)

In [4]:
#Get the dataset
import tensorflow_datasets as tfds

In [6]:
#Get list of dataset
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
#Load data set its 4GB Data
(train_data,test_data), ds_info = tfds.load(
    name = "food101",
    split = ["train","validation"],
    shuffle_files = True,
    as_supervised = True, #data gets returned in tuple formate(data, label)
    with_info = True

)